In [1]:
from datasets import load_dataset
from transformers import AutoProcessor, SeamlessM4TModel
import torch, time, numpy as np
import evaluate
from tqdm import tqdm

/home/kayleexie/miniconda3/envs/cs329e/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kayleexie/miniconda3/envs/cs329e/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [ ]:
device = 'cpu'
# Load model and processor
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium").to(device)
processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


: 

In [ ]:
# Load a small split of the CVSS English-Chinese test set
dataset = load_dataset("google/fleurs", "cmn_hans_cn", split="test[:1]")  # Chinese test samples

Generating train split: 997 examples [00:09, 123.49 examples/s]

In [ ]:
# Optional: Load BLEU scorer for intermediate text accuracy
bleu = evaluate.load("bleu")

In [ ]:
results = []

for sample in tqdm(dataset):
    audio = sample['audio']['array']
    sr = sample['audio']['sampling_rate']
    reference_text = sample['translation']['zh']

    inputs = processor(audios=audio, sampling_rate=sr, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run inference
    start = time.time()
    outputs = model.generate(**inputs, tgt_lang="cmn")
    end = time.time()

    predicted_audio = outputs[0].cpu().numpy().squeeze()
    input_dur = len(audio) / sr
    output_dur = len(predicted_audio) / 16000
    latency = end - start

    results.append({
        "input_duration": input_dur,
        "output_duration": output_dur,
        "latency": latency,
        "RTF": latency / input_dur,
    })

In [ ]:
# Display results
import pandas as pd
df = pd.DataFrame(results)
df.describe()